In [1]:
#for Google Colab
import sys
from google.colab import drive
from IPython.display import Javascript
from pprint import pprint

drive.mount('/content/drive')

###---- ROOT_PATHを記載 ----###
ROOT_PATH = '/content/drive/MyDrive/GENIAC/Scraping/'
sys.path.append(ROOT_PATH)

Mounted at /content/drive


In [2]:
!pip install -q trafilatura
!pip install -q beautifulsoup4
!pip install -q requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup

# スクレイピングする基本URL
base_url = 'https://sinri.net/page/'

# スクレイピングするURLのリスト
urls_to_scrape = ['https://sinri.net']
for i in range(2, 248):
  urls_to_scrape.append(base_url + str(i))

print(f"Number of URLs to scrape: {len(urls_to_scrape)}")

Number of URLs to scrape: 247


In [4]:
# 最終的なURLリストを保持するためのセット
final_urls = set()

# https://yuik.net/col/ から始まるページを取得する
# 各URLをループ処理
for url in urls_to_scrape:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a', href=True):
            href = link.get('href')
            if href.startswith('https://yuik.net/col/'):
                final_urls.add(href)
                print(href)  # 確認用にURLを表示
    else:
        print(f"Failed to retrieve content from {url}")

# 重複を削除したリストに変換
unique_urls = list(final_urls)

# ユニークなURLの数と内容を表示
print(f"Number of unique URLs: {len(unique_urls)}")
print(unique_urls)

https://yuik.net/col/51578.html
https://yuik.net/col/51578.html
https://yuik.net/col/51578.html
https://yuik.net/col/51574.html
https://yuik.net/col/51574.html
https://yuik.net/col/51574.html
https://yuik.net/col/51434.html
https://yuik.net/col/51434.html
https://yuik.net/col/51434.html
https://yuik.net/col/51430.html
https://yuik.net/col/51430.html
https://yuik.net/col/51430.html
https://yuik.net/col/51428.html
https://yuik.net/col/51428.html
https://yuik.net/col/51428.html
https://yuik.net/col/51057.html
https://yuik.net/col/51057.html
https://yuik.net/col/51057.html
https://yuik.net/col/51053.html
https://yuik.net/col/51053.html
https://yuik.net/col/51053.html
https://yuik.net/col/50927.html
https://yuik.net/col/50927.html
https://yuik.net/col/50927.html
https://yuik.net/col/50924.html
https://yuik.net/col/50924.html
https://yuik.net/col/50924.html
https://yuik.net/col/50922.html
https://yuik.net/col/50922.html
https://yuik.net/col/50922.html
https://yuik.net/col/50832.html
https://

In [5]:
urls = unique_urls
len(urls)

63

In [6]:
import requests
import csv, json, os, time
from bs4 import BeautifulSoup
from trafilatura import fetch_url, extract


output_dir = ROOT_PATH + "jsonl/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def fetch_all_pages_to_jsonl(jsonl_filename):
    filename = os.path.join(output_dir, jsonl_filename)
    # 抽出したページ数のカウント
    count = 0
    # 全ページ数のカウント
    total_count = 0

    with open(filename, 'w', encoding='utf_8_sig') as file:

      for url in urls:
        total_count += 1
        page_url = url
        downloaded = fetch_url(page_url)
        soup = BeautifulSoup(downloaded, 'html.parser')
        title = soup.title.text if soup.title else ''
        extracted_text = extract(downloaded, include_tables=False, target_lang='ja')

        # "。"を含まないページはフィルタリング（単語のみや英語のページが多いため）
        if not extracted_text or "。" not in extracted_text:
            # print(extracted_text)
            continue

        # jsonl形式で書き込み
        page_data = {'title': title, 'url': page_url, 'text': extracted_text}
        json.dump(page_data, file, ensure_ascii=False)
        file.write('\n')

        count += 1
        print(f"Processed {count} pages.") if count % 1000 == 0 else None
        # if total_count==10:
        #   break
        time.sleep(2)

    # 抽出したページ数を表示
    print(f"Total pages retrieved: {count}/ {total_count}")


In [7]:
fetch_all_pages_to_jsonl('sinri_net_02.jsonl')

Total pages retrieved: 63/ 63
